In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import pdb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from math import sqrt
from tensorflow.keras import layers, models
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:

def create_samples_sim(data, ph, hist, day_len_f):
    # import pdb
    """
    Create samples consisting in glucose, insulin and CHO histories (past hist-length values)
    :param data: dataframe
    :param ph: prediction horizon in minutes in sampling frequency scale
    :param hist: history length in sampling frequency scale
    :param day_len_f: length of day in sampling frequency scale
    :return: dataframe of samples
    """
    n_samples = data.shape[0] - ph - hist + 1
    # pdb.set_trace()
    y = data.loc[ph + hist - 1:, "glucose"].values.reshape(-1, 1)
    d = pd.DatetimeIndex(data.loc[ph + hist - 1:, "datetime"].values)
    # t = np.concatenate([np.arange(day_len_f) for _ in range(len(data) // day_len_f)], axis=0)[ph + hist - 1:].reshape(-1, 1)
    g = np.array([data.loc[i:i + n_samples - 1, "glucose"] for i in range(hist)]).transpose()
    c = np.array([data.loc[i:i + n_samples - 1, "CHO"] for i in range(hist)]).transpose()
    i = np.array([data.loc[i:i + n_samples - 1, "insulin"] for i in range(hist)]).transpose()

    new_columns = np.r_[["glucose_" + str(i) for i in range(hist)], ["CHO_" + str(i) for i in range(hist)], [
        "insulin_" + str(i) for i in range(hist)], ["y"]]
    new_data = pd.DataFrame(data=np.c_[g, c, i, y], columns=new_columns)
    new_data["datetime"] = d
    new_data = new_data.loc[:, np.r_[["datetime"], new_columns]]  # reorder the columns, with datetime first

    return new_data

def standardize_sim(data):
  columns = data.columns.drop(["datetime", 'y'])
  scaler = StandardScaler()

  # standardize the sets (-> ndarray) without datetime
  train = scaler.fit_transform(data.drop(["datetime", 'y'], axis=1))

  # recreate dataframe
  train = pd.DataFrame(data=train, columns=columns)

  # add datetime
  train["datetime"] = pd.DatetimeIndex(data.loc[:, "datetime"].values)
  train["y"] = data.loc[:, "y"].values

  # reorder
  train = train.loc[:, data.columns]

  train = train.reset_index(drop=True)

  return train

###Variables

In [ ]:
seed = 1  
cv = 5 
path = "." 
freq = 5
day_len = 1440
day_len_freq = day_len // freq
n_days_test = 10

params = {
    "hist": 180
}

search = {
    "lr": ["logarithmic", 3, 3],
}

hist = params["hist"] // freq


dataset='ohio'
subject='575'
ph=6

In [ ]:
def aggregate_inputs(data1):
    """
    Aggregate historical values to prepare inputs for the Hovorka model.
    :param data: preprocessed dataframe with 36 columns each for glucose, CHO, and insulin
    :return: dataframe with aggregated inputs for the Hovorka model
    """
    data = data1.copy()
    # List of historical columns
    glucose_cols = [f'glucose_{i}' for i in range(36)]
    cho_cols = [f'CHO_{i}' for i in range(36)]
    insulin_cols = [f'insulin_{i}' for i in range(36)]

    # Calculate the aggregated inputs
    data['glucose_input'] = data[glucose_cols].mean(axis=1)
    data['CHO_input'] = data[cho_cols].sum(axis=1)
    data['insulin_input'] = data[insulin_cols].sum(axis=1)

    # Select the relevant columns for the Hovorka model
    hovorka_inputs = data[['datetime', 'glucose_input', 'CHO_input', 'insulin_input', 'y']]

    return hovorka_inputs

In [ ]:
def scaling_range(data1):
    current_min = data1['CGM'].min()
    current_max = data1['CGM'].max()
    new_min = 39
    new_max = 450
    # Apply the linear transformation to scale the CGM values
    data1['CGM_scaled'] = ((data1['CGM'] - current_min) / (current_max - current_min)) * (new_max - new_min) + new_min
    data1.drop(columns=['CGM'], inplace=True)
    return data1

In [ ]:
data = pd.read_csv('/Users/narayaniverma/Documents/Masters_Dissertation/UVA/simglucose-master/results/results_23Aug_Final_1440/adult#009.csv', usecols = ['Time', 'CGM', 'CHO', 'insulin'])
data.drop(data.tail(1).index, inplace = True)
data = scaling_range(data)
data.rename(columns={'Time': 'datetime', 'CGM_scaled': 'glucose', 'CHO': 'CHO', 'insulin': 'insulin'}, inplace=True)

data_sampled_sim = create_samples_sim(data, ph, hist, day_len_freq)
data_hovorka_inputs_sim = aggregate_inputs(data_sampled_sim)

In [ ]:
G0 = 5
Bow = 70
k12 = 0.066
ka1 = 0.006
ka2 = 0.06
ka3 = 0.03
kb1 = 3.072 * 10**-5
kb2 = 4.92 * 10**-5
kb3 = 0.00156
ke = 0.138
VI = 0.12 * Bow
VG = 0.16 * Bow
AG = 0.8
tmax_G = 40
tmax_I = 55
SFIT = 51.2 * 10**-4
SFID = 8.2 * 10**-4
SFIE = 520 * 10**-4
EGP0 = 0.0161 * Bow
F01 = 0.0097 * Bow
Q10 = G0 * VG
u0 = 6.68
S10 = u0 * tmax_I
S20 = S10
Ui0 = S20/tmax_I
I0 = Ui0/(ke * VI)
x10 = SFIT * I0
x20 = SFID * I0
x30 = SFIE * I0
Q20 = x10 / (x20 + k12)

preprocessed_data_w_hovorka_sim = []


# Function to define Hovorka model equations
def hovorka_model(y, t, d, u):
    Q1, Q2, x1, x2, x3, D1, D2, S1, S2, I = y

    # Glucose absorption compartments
    D = 1000 * 180.156 * np.interp(t, np.arange(len(d)), d)
    dD1_dt = AG * D - D1 / tmax_G
    dD2_dt = D1 / tmax_G - D2 / tmax_G
    UG = D2 / tmax_G

    # Amount of glucose
    F01_c = F01 if Q1 / VG >= 4.5 else F01 * (Q1 / VG / 4.5)
    FR = 0.003 * (Q1 / VG - 9) * VG if Q1 / VG >= 9 else 0

    # Amount of glucose
    dQ1_dt = UG - x1 * Q1 - F01_c - FR + k12 * Q2 + EGP0 * (1 - x3)
    dQ2_dt = x1 * Q1 - (k12 + x2) * Q2
    G = Q1 / VG

    # Insulin effects
    dx1_dt = -ka1 * x1 + kb1 * I
    dx2_dt = -ka2 * x2 + kb2 * I
    dx3_dt = -ka3 * x3 + kb3 * I

    # Insulin absorption compartments
    dS1_dt = np.interp(t, np.arange(len(u)), u) - S1 / tmax_I
    dS2_dt = S1 / tmax_I - S2 / tmax_I
    dI_dt = S2 / tmax_I / VI - ke * I

    return [dQ1_dt, dQ2_dt, dx1_dt, dx2_dt, dx3_dt, dD1_dt, dD2_dt, dS1_dt, dS2_dt, dI_dt]

for k,j in [(data_hovorka_inputs_sim, data_sampled_sim)]:
  glucose_avg = k['glucose_input'].values
  CHO_sum = k['CHO_input'].values
  insulin_sum = k['insulin_input'].values

  # Define the time points (assuming they are evenly spaced)
  time_points = np.arange(len(k))
  initial_conditions = [Q10, Q20, x10, x20, x30, 0, 0, S10, S20, I0]

  # Integrate the Hovorka model equations over time
  results = odeint(hovorka_model, initial_conditions, time_points, args=(CHO_sum, insulin_sum))

  # Extracting features
  G_t = results[:, 0] / VG
  I_t = results[:, -1]
  UG_t = results[:, 6] / tmax_G

  # Create a dataframe with the results
  hovorka_features = pd.DataFrame({
      'G_t': G_t,
      'I_t': I_t,
      'UG_t': UG_t
  }, index=j.index)

  preprocessed_data_w_hovorka_sim = pd.concat([j, hovorka_features], axis=1)

In [ ]:
k = preprocessed_data_w_hovorka_sim
l = data_sampled_sim
l_g = pd.DataFrame()
l_c = pd.DataFrame()
l_i = pd.DataFrame()
l_y = pd.DataFrame()

n_samples = k.shape[0] - ph - hist + 1

g = np.array([k.loc[i:i + n_samples - 1, "G_t"] for i in range(hist)]).transpose()
i_n = np.array([k.loc[i:i + n_samples - 1, "I_t"] for i in range(hist)]).transpose()
c = np.array([k.loc[i:i + n_samples - 1, "UG_t"] for i in range(hist)]).transpose()

new_columns = np.r_[["G_t" + str(i) for i in range(hist)], ["I_t" + str(i) for i in range(hist)], [
      "UG_t" + str(i) for i in range(hist)]]
new_data = pd.DataFrame(data=np.c_[g, i_n, c], columns=new_columns)
# Assuming 'train' is your list of DataFrames with original sequential data
# Merge Hovorka sequences with each DataFrame in train

l_g = l.loc[:, :'glucose_35']
l_c = l.loc[:, 'CHO_0':'CHO_35']
l_i = l.loc[:, 'insulin_0':'insulin_35']
l_y = l.loc[:, 'y']
new_data_g = new_data.loc[:, 'G_t0':'G_t35']
new_data_c = new_data.loc[:, 'UG_t0':'UG_t35']
new_data_i = new_data.loc[:, 'I_t0':'I_t35']
l = pd.concat([l_g, new_data_g.iloc[:len(l)].reset_index(drop=True), l_c, new_data_c.iloc[:len(l)].reset_index(drop=True), l_i, new_data_i.iloc[:len(l)].reset_index(drop=True), l_y], axis=1)
list_of_cols = l.loc[:0, 'G_t0':'UG_t35'].columns  # retrieve only the 0th row for efficiency
l.dropna(subset = list_of_cols, inplace = True)

data_sampled_sim = l


In [ ]:
train = standardize_sim(data_sampled_sim)

X_train_sim = train.drop(['y', 'datetime'], axis=1).values
y_train_sim = train['y'].values

X_train_sim = X_train_sim.reshape(-1, 36, 6)
y_train_sim = y_train_sim.reshape(-1, 1)

In [ ]:
def positional_encoding(seq_len, d_model):
    PE = np.zeros((seq_len, d_model))
    for pos in range(seq_len):
        for i in range(0, d_model, 2):
            PE[pos, i] = np.sin(pos / (10000 ** ((2 * i) / d_model)))
            if i + 1 < d_model:
                PE[pos, i + 1] = np.cos(pos / (10000 ** ((2 * (i + 1)) / d_model)))
    return tf.constant(PE, dtype=tf.float32)

# d_model = len(features)  # This should match the number of input features
pos_encoding = positional_encoding(36, 6)

In [ ]:
def transformer_model(input_shape, d_model, num_heads, num_layers, dropout=0.1):
    inputs = layers.Input(shape=input_shape)
    x = inputs + pos_encoding[:input_shape[0], :]

    for _ in range(num_layers):
        x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x)
        x = layers.Dropout(dropout)(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x)

        x_ff = layers.Dense(d_model, activation='relu')(x)
        x_ff = layers.Dropout(dropout)(x_ff)
        x = layers.LayerNormalization(epsilon=1e-6)(x_ff + x)

    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(1)(x)

    model = models.Model(inputs, outputs)
    return model

input_shape = (36, 6)
d_model = 6
num_heads = 8
num_layers = 3
dropout = 0.1

model = transformer_model(input_shape, d_model, num_heads, num_layers, dropout)
model.compile(optimizer='adam', loss='mse',
              metrics=[tf.keras.metrics.RootMeanSquaredError(), 'mape'])

early_stopping = EarlyStopping(monitor='loss',  # you can also use 'val_rmse' or any other metric you're monitoring
                               patience=10,         # number of epochs with no improvement after which training will be stopped
                               restore_best_weights=True,  # restores model weights from the epoch with the best value of the monitored metric
                               verbose=1)

early_stopping_real = EarlyStopping(monitor='loss',
                                    patience=10,
                                    restore_best_weights=True,
                                    verbose=1)

# Define ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(monitor='loss',   
                              factor=0.5,           # factor by which the learning rate will be reduced
                              patience=5,           # number of epochs with no improvement after which learning rate will be reduced
                              min_lr=1e-6,          # lower bound on the learning rate
                              verbose=1)

model.summary()

In [ ]:
history_sim = model.fit(X_train_sim, y_train_sim, epochs=20, batch_size=32, callbacks=[early_stopping, reduce_lr])

In [ ]:
model.save_weights('model_sim.weights.h5')

##Preprocessing

###Cleaning

In [ ]:
import numpy as np


def fill_nans(data, day_len, n_days_test):
    """
    Fill NaNs inside the dataframe of samples following:
    - CHO and insulin values are filled with 0
    - glucose history are interpolated (linearly) when possible and extrapolated if not
    :param data: sample dataframe
    :param day_len: length of day, scaled to sampling frequency
    :param n_days_test: number of test days
    :return: cleaned sample dataframe
    """
    data_nan = data.copy()

    # fill insulin and CHO nans with 0
    for col in data.columns:
        if "insulin" in col or "CHO" in col:
            data[col] = data[col].fillna(0)

    # fill glucose nans
    g_col = [col for col in data.columns if "glucose" in col]
    g_mean = data.y.mean()
    for i in range(len(data.index)):
        g_i = data.loc[i, g_col]
        if g_i.notna().all():  # no nan
            pass
        elif g_i.isna().all():  # all nan
            if i > len(data) - n_days_test * day_len + 1:
                last_known_gs = data_nan.loc[:i - 1, "glucose_0"][data_nan.loc[:i - 1, "glucose_0"].notna()]
                if len(last_known_gs) >= 1:
                    for col in g_col:
                        # data.loc[i,col] = last_known_gs.iloc[-1]
                        data.loc[i, col] = g_mean
                else:
                    for col in g_col:
                        data.loc[i, col] = g_mean
        else:  # some nan
            # compute insample nan indices, and last outsample + insample nonnan indices
            isna_idx = i + np.where(g_i.isna())[0]
            notna_idx = i + np.where(g_i.notna())[0]
            if data_nan.loc[:i - 1, "glucose_0"].notna().any():
                mask = data_nan.loc[:i - 1, "glucose_0"].notna().values
                notna_idx = np.r_[data_nan.loc[:i - 1, "glucose_0"][mask].index[-2:], notna_idx]

            # for all nan
            for isna_i in isna_idx:
                # get the two closest non nan values
                idx_diff = notna_idx - isna_i

                if np.any(idx_diff > 0) and np.any(idx_diff < 0):
                    # we got a start and an end
                    start = notna_idx[np.where(idx_diff < 0, idx_diff, -np.inf).argmax()]
                    end = notna_idx[np.where(idx_diff > 0, idx_diff, np.inf).argmin()]
                    start_val, end_val = data_nan.loc[start, "glucose_0"], data_nan.loc[end, "glucose_0"]

                    # interpolate between them
                    rate = (end_val - start_val) / (end - start)
                    data.loc[i, g_col[isna_i - i]] = data_nan.loc[start, "glucose_0"] + rate * (isna_i - start)
                elif np.any(idx_diff > 0):
                    # we only have end(s)
                    # backward extrapolation - only used in very first day where there is no start
                    if len(idx_diff) >= 2:
                        # we have two last values so we can compute a rate
                        end1, end2 = notna_idx[0], notna_idx[1]
                        end1_val, end2_val = data_nan.loc[end1, "glucose_0"], data_nan.loc[end2, "glucose_0"]
                        rate = (end2_val - end1_val) / (end2 - end1)
                        data.loc[i, g_col[isna_i - i]] = data_nan.loc[end1, "glucose_0"] - rate * (end1 - isna_i)
                    else:
                        # we have only one value so we cannot compute a rate
                        end = notna_idx[0]
                        end_val = data_nan.loc[end, "glucose_0"]
                        data.loc[i, g_col[isna_i - i]] = end_val
                elif np.any(idx_diff < 0):
                    last_val = g_i[g_i.notna()][-1]
                    data.loc[i, g_col[isna_i - i]] = last_val

    return data


def remove_nans(data):
    """
    Remove samples that still have NaNs (in y column mostly)
    :param data: dataframe of samples
    :return: no-NaN dataframe
    """
    new_data = []
    for df in data:
        new_data.append(df.dropna())
    return new_data

###Loading

In [ ]:
import os
import xml.etree.ElementTree as ET
import pandas as pd


def load_ohio(dataset, subject):
    """
    Load OhioT1DM training_old and testing files into a dataframe
    :param dataset: name of dataset
    :param subject: name of subject
    :return: dataframe
    """
    train_path, test_path = _compute_file_names(dataset, subject)

    [train_xml, test_xml] = [ET.parse(set).getroot() for set in [train_path, test_path]]

    [train, test] = [_extract_data_from_xml(xml) for xml in [train_xml, test_xml]]

    data = pd.concat([train, test], ignore_index=True)

    return data


def _compute_file_names(dataset, subject):
    """
    Compute the name of the files, given dataset and subject
    :param dataset: name of dataset
    :param subject: name of subject
    :return: path to training file, path to testing file
    """
    train_dir = "/Users/narayaniverma/Documents/Masters_Dissertation/OhioT1DM/2018/train"
    train_file = subject + "-ws-training.xml"
    train_path = os.path.join(train_dir, train_file)

    test_dir = "/Users/narayaniverma/Documents/Masters_Dissertation/OhioT1DM/2018/test"
    test_file = subject + "-ws-testing.xml"
    test_path = os.path.join(test_dir, test_file)

    return train_path, test_path


def _extract_data_from_xml(xml):
    """
    extract glucose, CHO, and insulin from xml and merge the data
    :param xml:
    :return: dataframe
    """
    glucose_df = _get_glucose_from_xml(xml)
    CHO_df = _get_CHO_from_xml(xml)
    insulin_df = _get_insulin_from_xml(xml)

    print("glocose:", glucose_df)
    print("CHO:", CHO_df)
    print("insulin:", insulin_df)

    df = pd.merge(glucose_df, CHO_df, how="outer", on="datetime")
    df = pd.merge(df, insulin_df, how="outer", on="datetime")
    df = df.sort_values("datetime")

    return df


def _get_field_labels(etree, field_index):
    """
    extract labels from xml tree
    :param etree: etree
    :param field_index:  position of field
    :return:
    """
    print("list of field labels:", list(etree[field_index][0].attrib.keys()))
    return list(etree[field_index][0].attrib.keys())


def _iter_fields(etree, field_index):
    """
    extract columns inside xml tree
    :param etree: tree
    :param field_index: position of columns
    :return:
    """
    for event in etree[field_index].iter("event"):
        yield list(event.attrib.values())


def _get_CHO_from_xml(xml):
    """
    Extract CHO values from xml
    :param xml:
    :return: CHO dataframe
    """
    labels = _get_field_labels(xml, field_index=5)
    CHO = list(_iter_fields(xml, field_index=5))
    CHO_df = pd.DataFrame(data=CHO, columns=labels)
    CHO_df.drop("type", axis=1, inplace=True)
    CHO_df["ts"] = pd.to_datetime(CHO_df["ts"], format="%d-%m-%Y %H:%M:%S")
    CHO_df["carbs"] = CHO_df["carbs"].astype("float")
    CHO_df.rename(columns={'ts': 'datetime', 'carbs': 'CHO'}, inplace=True)
    return CHO_df


def _get_insulin_from_xml(xml):
    """
    Extract insulin values from xml
    :param xml:
    :return: insulin dataframe
    """
    labels = _get_field_labels(xml, field_index=4)
    insulin = list(_iter_fields(xml, field_index=4))
    insulin_df = pd.DataFrame(data=insulin, columns=labels)
    for col in ["ts_end", "type", "bwz_carb_input"]:
        insulin_df.drop(col, axis=1, inplace=True)
    insulin_df["ts_begin"] = pd.to_datetime(insulin_df["ts_begin"], format="%d-%m-%Y %H:%M:%S")
    insulin_df["dose"] = insulin_df["dose"].astype("float")
    insulin_df.rename(columns={'ts_begin': 'datetime', 'dose': 'insulin'}, inplace=True)
    return insulin_df


def _get_glucose_from_xml(xml):
    """
    Extract glucose values from xml
    :param xml:
    :return: glucose dataframe
    """
    labels = _get_field_labels(xml, field_index=0)
    glucose = list(_iter_fields(xml, field_index=0))
    glucose_df = pd.DataFrame(data=glucose, columns=labels)
    glucose_df["ts"] = pd.to_datetime(glucose_df["ts"], format="%d-%m-%Y %H:%M:%S")
    glucose_df["value"] = glucose_df["value"].astype("float")
    glucose_df.rename(columns={'ts': 'datetime', 'value': 'glucose'}, inplace=True)
    return glucose_df

###Resampling

In [ ]:
from datetime import timedelta, datetime
def resample(data, freq):
    """
    :param data: dataframe
    :param freq: sampling frequency
    :return: resampled data between the the first day at 00:00:00 and the last day at 23:60-freq:00 at freq sample frequency
    """
    start = data.datetime.iloc[0].strftime('%Y-%m-%d') + " 00:00:00"
    end = datetime.strptime(data.datetime.iloc[-1].strftime('%Y-%m-%d'), "%Y-%m-%d") + timedelta(days=1) - timedelta(
        minutes=freq)
    index = pd.period_range(start=start,
                            end=end,
                            freq=str(freq) + 'min').to_timestamp()
    data = data.resample(str(freq) + 'min', on="datetime").agg({'glucose': np.mean, 'CHO': np.sum, "insulin": np.sum})
    data = data.reindex(index=index)
    data = data.reset_index()
    data = data.rename(columns={"index": "datetime"})
    return data

###Create Samples

In [ ]:
def create_samples(data, ph, hist, day_len_f):
    """
    Create samples consisting in glucose, insulin and CHO histories (past hist-length values)
    :param data: dataframe
    :param ph: prediction horizon in minutes in sampling frequency scale
    :param hist: history length in sampling frequency scale
    :param day_len_f: length of day in sampling frequency scale
    :return: dataframe of samples
    """
    n_samples = data.shape[0] - ph - hist + 1

    y = data.loc[ph + hist - 1:, "glucose"].values.reshape(-1, 1)
    d = pd.DatetimeIndex(data.loc[ph + hist - 1:, "datetime"].values)
    t = np.concatenate([np.arange(day_len_f) for _ in range(len(data) // day_len_f)], axis=0)[ph + hist - 1:].reshape(-1, 1)
    g = np.array([data.loc[i:i + n_samples - 1, "glucose"] for i in range(hist)]).transpose()
    c = np.array([data.loc[i:i + n_samples - 1, "CHO"] for i in range(hist)]).transpose()
    i = np.array([data.loc[i:i + n_samples - 1, "insulin"] for i in range(hist)]).transpose()

    new_columns = np.r_[["time"], ["glucose_" + str(i) for i in range(hist)], ["CHO_" + str(i) for i in range(hist)], [
        "insulin_" + str(i) for i in range(hist)], ["y"]]
    new_data = pd.DataFrame(data=np.c_[t, g, c, i, y], columns=new_columns)
    new_data["datetime"] = d
    new_data = new_data.loc[:, np.r_[["datetime"], new_columns]]  # reorder the columns, with datetime first

    return new_data

###Remove/Fill Nans

In [ ]:


def remove_nans(data):
    """
    Remove samples that still have NaNs (in y column mostly)
    :param data: dataframe of samples
    :return: no-NaN dataframe
    """
    new_data = []
    for df in data:
        new_data.append(df.dropna())
    return new_data

def fill_nans(data, day_len, n_days_test):
    """
    Fill NaNs inside the dataframe of samples following:
    - CHO and insulin values are filled with 0
    - glucose history are interpolated (linearly) when possible and extrapolated if not
    :param data: sample dataframe
    :param day_len: length of day, scaled to sampling frequency
    :param n_days_test: number of test days
    :return: cleaned sample dataframe
    """
    data_nan = data.copy()

    # fill insulin and CHO nans with 0
    for col in data.columns:
        if "insulin" in col or "CHO" in col:
            data[col] = data[col].fillna(0)

    # fill glucose nans
    g_col = [col for col in data.columns if "glucose" in col]
    g_mean = data.y.mean()
    for i in range(len(data.index)):
        g_i = data.loc[i, g_col]
        if g_i.notna().all():  # no nan
            pass
        elif g_i.isna().all():  # all nan
            if i > len(data) - n_days_test * day_len + 1:
                last_known_gs = data_nan.loc[:i - 1, "glucose_0"][data_nan.loc[:i - 1, "glucose_0"].notna()]
                if len(last_known_gs) >= 1:
                    for col in g_col:
                        # data.loc[i,col] = last_known_gs.iloc[-1]
                        data.loc[i, col] = g_mean
                else:
                    for col in g_col:
                        data.loc[i, col] = g_mean
        else:  # some nan
            # compute insample nan indices, and last outsample + insample nonnan indices
            isna_idx = i + np.where(g_i.isna())[0]
            notna_idx = i + np.where(g_i.notna())[0]
            if data_nan.loc[:i - 1, "glucose_0"].notna().any():
                mask = data_nan.loc[:i - 1, "glucose_0"].notna().values
                notna_idx = np.r_[data_nan.loc[:i - 1, "glucose_0"][mask].index[-2:], notna_idx]

            # for all nan
            for isna_i in isna_idx:
                # get the two closest non nan values
                idx_diff = notna_idx - isna_i

                if np.any(idx_diff > 0) and np.any(idx_diff < 0):
                    # we got a start and an end
                    start = notna_idx[np.where(idx_diff < 0, idx_diff, -np.inf).argmax()]
                    end = notna_idx[np.where(idx_diff > 0, idx_diff, np.inf).argmin()]

                    start_idx = _compute_indexes(i, start, len(data_nan))
                    end_idx = _compute_indexes(i, end, len(data_nan))

                    start_val = data_nan.loc[start_idx]
                    end_val = data_nan.loc[end_idx]

                    # interpolate between them
                    rate = (end_val - start_val) / (end - start)
                    data.loc[i, g_col[isna_i - i]] = data_nan.loc[start_idx] + rate * (isna_i - start)
                elif np.any(idx_diff > 0):
                    # we only have end(s)
                    # backward extrapolation - only used in very first day where there is no start
                    if len(idx_diff) >= 2:
                        # we have two last values so we can compute a rate
                        end1, end2 = notna_idx[0], notna_idx[1]
                        [end1_idx, end2_idx] = [_compute_indexes(i, _, len(data_nan)) for _ in [end1, end2]]
                        end1_val, end2_val = data_nan.loc[end1_idx], data_nan.loc[end2_idx]
                        rate = (end2_val - end1_val) / (end2 - end1)
                        data.loc[i, g_col[isna_i - i]] = data_nan.loc[end1_idx] - rate * (end1 - isna_i)
                    else:
                        # we have only one value so we cannot compute a rate
                        end = notna_idx[0]
                        end_idx = _compute_indexes(i, end, len(data_nan))
                        end_val = data_nan.loc[end_idx]
                        data.loc[i, g_col[isna_i - i]] = end_val
                elif np.any(idx_diff < 0):
                    # forward extrapolation
                    if len(idx_diff) >= 2:
                        end1, end2 = notna_idx[-2], notna_idx[-1]
                        [end1_idx, end2_idx] = [_compute_indexes(i, _, len(data_nan)) for _ in [end1, end2]]
                        end1_val, end2_val = data_nan.loc[end1_idx], data_nan.loc[end2_idx]
                        rate = (end2_val - end1_val) / (end2 - end1)
                        data.loc[i, g_col[isna_i - i]] = data_nan.loc[end1_idx] - rate * (end1 - isna_i)
                    else:
                        # we only have one value, so we cannot compute a rate
                        last_val = g_i[g_i.notna()][-1]
                        data.loc[i, g_col[isna_i - i]] = last_val

    return data


def _compute_indexes(i, index, len):
    if index >= len:
        return (i, "glucose_" + str(index - i))
    else:
        return (index, "glucose_0")

###Splitting

In [ ]:
import numpy as np
import pandas as pd
import sklearn.model_selection as sk_model_selection

def split(data, day_len, test_n_days, cv_factor):
    """
    Split samples into training, validation, and testing days. Testing days are the last test_n_days days, and training
    and validation sets are created by permuting of splits according to the cv_factor.
    :param data: dataframe of samples
    :param day_len: length of day in freq minutes
    :param test_n_days: number of testing days
    :param cv_factor: cross validation factor
    :return: training, validation, testing samples folds
    """
    # the test split is equal to the last test_n_days days of data
    test = [data.iloc[-test_n_days * day_len:].copy()]

    # train+valid samples = all minus first and test days
    fday_n_samples = data.shape[0] - (data.shape[0] // day_len * day_len)
    train_valid = data.iloc[fday_n_samples:-test_n_days * day_len].copy()

    # split train_valid into cv_factor folds for inner cross-validation
    n_days = train_valid.shape[0] // day_len
    days = np.arange(n_days)

    kf = sk_model_selection.KFold(n_splits=cv_factor, shuffle=True, random_state=seed)

    train, valid = [], []
    for train_idx, valid_idx in kf.split(days):
        def get_whole_day(data, i):
            return data[i * day_len:(i + 1) * day_len]

        train.append(pd.concat([get_whole_day(train_valid, i) for i in train_idx], axis=0, ignore_index=True))
        valid.append(pd.concat([get_whole_day(train_valid, i) for i in valid_idx], axis=0, ignore_index=True))

    return train, valid, test

###Standardization

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

def standardize(train, valid, test):
    """
    Standardize (zero mean and unit variance) the sets w.r.t to the training set for every fold
    :param train: training sample fold
    :param valid: validation sample fold
    :param test: testing sample fold
    :return: standardized training, validation, and testing sets
    """
    columns = train[0].columns.drop(["datetime", 'y'])
    train_scaled, valid_scaled, test_scaled, scalers = [], [], [], []
    for i in range(len(train)):
        scaler = StandardScaler()

        # standardize the sets (-> ndarray) without datetime
        train_i = scaler.fit_transform(train[i].drop(["datetime", 'y'], axis=1))
        valid_i = scaler.transform(valid[i].drop(["datetime", 'y'], axis=1))
        test_i = scaler.transform(test[0].copy().drop(["datetime", 'y'], axis=1))

        # recreate dataframe
        train_i = pd.DataFrame(data=train_i, columns=columns)
        valid_i = pd.DataFrame(data=valid_i, columns=columns)
        test_i = pd.DataFrame(data=test_i, columns=columns)

        # add datetime
        train_i["datetime"] = pd.DatetimeIndex(train[i].loc[:, "datetime"].values)
        train_i["y"] = train[i].loc[:, "y"].values
        valid_i["datetime"] = pd.DatetimeIndex(valid[i].loc[:, "datetime"].values)
        valid_i["y"] = valid[i].loc[:, "y"].values
        test_i["datetime"] = pd.DatetimeIndex(test[0].loc[:, "datetime"].values)
        test_i["y"] = test[0].loc[:, "y"].values

        # reorder
        train_i = train_i.loc[:, train[i].columns]
        valid_i = valid_i.loc[:, valid[i].columns]
        test_i = test_i.loc[:, test[0].columns]

        # save
        train_scaled.append(train_i)
        valid_scaled.append(valid_i)
        test_scaled.append(test_i)

        scalers.append(scaler)

    return train_scaled, valid_scaled, test_scaled, scalers

In [ ]:
def test_dup(test, train):
  columns = train[0].columns.drop(["datetime", 'y'])
  test_scaled = []
  for i in range(len(train)):

    # standardize the sets (-> ndarray) without datetime
    test_i = test[0].copy().drop(["datetime", 'y'], axis=1)

    # recreate dataframe
    test_i = pd.DataFrame(data=test_i, columns=columns)

    # add datetime
    test_i["datetime"] = pd.DatetimeIndex(test[0].loc[:, "datetime"].values)
    test_i["y"] = test[0].loc[:, "y"].values

    # reorder
    test_i = test_i.loc[:, test[0].columns]

    test_i = test_i.reset_index(drop=True)

    # save
    test_scaled.append(test_i)

  return test_scaled

###Main

In [ ]:
data = load_ohio(dataset, subject)
data_resampled = resample(data, freq)
data_sampled = create_samples(data_resampled, ph, hist, day_len_freq)
data_filled_na = fill_nans(data_sampled, day_len_freq, n_days_test)
train, valid, test = split(data_filled_na, day_len_freq, n_days_test, cv)
[train, valid, test] = [remove_nans(set) for set in [train, valid, test]]
test = test_dup(test, train)

# train, valid, test, scalers = standardize(train, valid, test)

In [ ]:
Initial_features_train = train[0].columns
Initial_features_valid = valid[0].columns
Initial_features_test = test[0].columns

##Models

In [ ]:
hovorka_inputs_train = []
for i in range(len(train)):
  hovorka_inputs_train.append(aggregate_inputs(train[i]))

hovorka_inputs_valid = []
for i in range(len(valid)):
  hovorka_inputs_valid.append(aggregate_inputs(valid[i]))

hovorka_inputs_test = []
for i in range(len(test)):
  hovorka_inputs_test.append(aggregate_inputs(test[i]))

In [ ]:
# Constants from Hovorka model
G0 = 5
Bow = 70
k12 = 0.066
ka1 = 0.006
ka2 = 0.06
ka3 = 0.03
kb1 = 3.072 * 10**-5
kb2 = 4.92 * 10**-5
kb3 = 0.00156
ke = 0.138
VI = 0.12 * Bow
VG = 0.16 * Bow
AG = 0.8
tmax_G = 40
tmax_I = 55
SFIT = 51.2 * 10**-4
SFID = 8.2 * 10**-4
SFIE = 520 * 10**-4
EGP0 = 0.0161 * Bow
F01 = 0.0097 * Bow
Q10 = G0 * VG
u0 = 6.68
S10 = u0 * tmax_I
S20 = S10
Ui0 = S20/tmax_I
I0 = Ui0/(ke * VI)
x10 = SFIT * I0
x20 = SFID * I0
x30 = SFIE * I0
Q20 = x10 / (x20 + k12)

preprocessed_data_w_hovorka_train = []
preprocessed_data_w_hovorka_valid = []
preprocessed_data_w_hovorka_test = []


# Function to define Hovorka model equations
def hovorka_model(y, t, d, u):
    Q1, Q2, x1, x2, x3, D1, D2, S1, S2, I = y

    # Glucose absorption compartments
    D = 1000 * 180.156 * np.interp(t, np.arange(len(d)), d)
    dD1_dt = AG * D - D1 / tmax_G
    dD2_dt = D1 / tmax_G - D2 / tmax_G
    UG = D2 / tmax_G

    # Amount of glucose
    F01_c = F01 if Q1 / VG >= 4.5 else F01 * (Q1 / VG / 4.5)
    FR = 0.003 * (Q1 / VG - 9) * VG if Q1 / VG >= 9 else 0

    # Amount of glucose
    dQ1_dt = UG - x1 * Q1 - F01_c - FR + k12 * Q2 + EGP0 * (1 - x3)
    dQ2_dt = x1 * Q1 - (k12 + x2) * Q2
    G = Q1 / VG

    # Insulin effects
    dx1_dt = -ka1 * x1 + kb1 * I
    dx2_dt = -ka2 * x2 + kb2 * I
    dx3_dt = -ka3 * x3 + kb3 * I

    # Insulin absorption compartments
    dS1_dt = np.interp(t, np.arange(len(u)), u) - S1 / tmax_I
    dS2_dt = S1 / tmax_I - S2 / tmax_I
    dI_dt = S2 / tmax_I / VI - ke * I

    return [dQ1_dt, dQ2_dt, dx1_dt, dx2_dt, dx3_dt, dD1_dt, dD2_dt, dS1_dt, dS2_dt, dI_dt]
# Extract the relevant columns
# flag =0
for (k,j) in ((hovorka_inputs_train, train), (hovorka_inputs_valid, valid), (hovorka_inputs_test, test)):
  for i in range(len(k)):
    # print(i)
    # if k is hovorka_inputs_valid and i == 2:
    #   flag = 1
    glucose_avg = k[i]['glucose_input'].values
    CHO_sum = k[i]['CHO_input'].values
    insulin_sum = k[i]['insulin_input'].values

    # Define the time points (assuming they are evenly spaced)
    time_points = np.arange(len(k[i]))
    initial_conditions = [Q10, Q20, x10, x20, x30, 0, 0, S10, S20, I0]

    # pdb.set_trace()
    # Integrate the Hovorka model equations over time
    results = odeint(hovorka_model, initial_conditions, time_points, args=(CHO_sum, insulin_sum))

    # Extracting features
    G_t = results[:, 0] / VG
    I_t = results[:, -1]
    UG_t = results[:, 6] / tmax_G

    # Create a dataframe with the results
    hovorka_features = pd.DataFrame({
        'G_t': G_t,
        'I_t': I_t,
        'UG_t': UG_t
    }, index=j[i].index)

    # print(hovorka_features)
    # print(train[i])
    if j is train:
      # Combine with preprocessed data if needed
      preprocessed_data_w_hovorka_train.append(pd.concat([j[i], hovorka_features], axis=1))
      # print(preprocessed_data_with_hovorka)
    elif j is valid:
      preprocessed_data_w_hovorka_valid.append(pd.concat([j[i], hovorka_features], axis=1))
    else:
      preprocessed_data_w_hovorka_test.append(pd.concat([j[i], hovorka_features], axis=1))

# print(preprocessed_data_w_hovorka_train[0])
# print(preprocessed_data_w_hovorka_valid[0])
# print(preprocessed_data_w_hovorka_test[0])

In [ ]:
for (k,l) in ((preprocessed_data_w_hovorka_train, train), (preprocessed_data_w_hovorka_valid, valid), (preprocessed_data_w_hovorka_test, test)):

  l_g = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]
  l_c = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]
  l_i = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]
  l_y = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]
  for j in range(len(k)):

    n_samples = k[j].shape[0] - ph - hist + 1

    g = np.array([k[j].loc[i:i + n_samples - 1, "G_t"] for i in range(hist)]).transpose()
    i_n = np.array([k[j].loc[i:i + n_samples - 1, "I_t"] for i in range(hist)]).transpose()
    c = np.array([k[j].loc[i:i + n_samples - 1, "UG_t"] for i in range(hist)]).transpose()

    new_columns = np.r_[["G_t" + str(i) for i in range(hist)], ["I_t" + str(i) for i in range(hist)], [
          "UG_t" + str(i) for i in range(hist)]]
    new_data = pd.DataFrame(data=np.c_[g, i_n, c], columns=new_columns)
    # Assuming 'train' is your list of DataFrames with original sequential data
    # Merge Hovorka sequences with each DataFrame in train
    # pdb.set_trace()
    l_g[j] = l[j].loc[:, :'glucose_35']
    l_c[j] = l[j].loc[:, 'CHO_0':'CHO_35']
    l_i[j] = l[j].loc[:, 'insulin_0':'insulin_35']
    l_y[j] = l[j].loc[:, 'y']
    new_data_g = new_data.loc[:, 'G_t0':'G_t35']
    new_data_c = new_data.loc[:, 'UG_t0':'UG_t35']
    new_data_i = new_data.loc[:, 'I_t0':'I_t35']
    l[j] = pd.concat([l_g[j], new_data_g.iloc[:len(l[j])].reset_index(drop=True), l_c[j], new_data_c.iloc[:len(l[j])].reset_index(drop=True), l_i[j], new_data_i.iloc[:len(l[j])].reset_index(drop=True), l_y[j]], axis=1)
    list_of_cols = l[j].loc[:0, 'G_t0':'UG_t35'].columns  # retrieve only the 0th row for efficiency
    l[j].dropna(subset = list_of_cols, inplace = True)
# Now each DataFrame in 'train' includes the original sequences and the new Hovorka model feature sequences


In [ ]:
from sklearn.preprocessing import StandardScaler
# # Combine all your training data into a single DataFrame

# Initialize the scaler
scaler = StandardScaler()

train, valid, test, scalers = standardize(train, valid, test)

train_data = train[0]
valid_data = valid[0]
train_data = pd.concat([train_data, valid_data], ignore_index=True)
test_data = test[0]

In [ ]:
X_train = train_data.drop(['y', 'datetime', 'time'], axis=1).values
y_train = train_data['y'].values

X_train = X_train.reshape(-1, 36, 6)
y_train = y_train.reshape(-1, 1)

X_test = test_data.drop(['y', 'datetime', 'time'], axis=1).values
y_test = test_data['y'].values

X_test = X_test.reshape(-1, 36, 6)
y_test = y_test.reshape(-1, 1)

###after tuner

In [ ]:
model.load_weights('model_sim.weights.h5')

In [ ]:
history_real = model.fit(X_train, y_train, epochs=20, batch_size=32, callbacks=[early_stopping_real, reduce_lr])

In [ ]:
model.save_weights('model_finetuned.weights.h5')

In [ ]:
model.load_weights('model_finetuned.weights.h5')

In [ ]:
import matplotlib.pyplot as plt



y_pred = model.predict(X_test)
# Evaluate
test_loss, test_rmse, test_mape = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}', f'Test MAE: {test_mape}', f'Test RMSE: {test_rmse}')

# Plotting the results
plt.figure(figsize=(30, 6))
plt.plot(y_test, color='blue', label='Actual BG')
plt.plot(y_pred, color='red', label='Predicted BG')
plt.title('BG Prediction')
plt.xlabel('Time')
plt.ylabel('BG')
plt.legend()
plt.show()


In [ ]:
# Convert 'datetime' column to pandas datetime format if it's not already
test_data['datetime'] = pd.to_datetime(test_data['datetime'])
# train_data
 

# Filter the data for the date '20-10-2021'
date_filter = '2022-01-06'
filtered_data = test_data[test_data['datetime'].dt.strftime('%Y-%m-%d') == date_filter]
# filtered_data

# # Extract the filtered actual and predicted BG levels
y_test_filtered = y_test[filtered_data.index]
y_pred_filtered = y_pred[filtered_data.index]

# # Plotting the results for the filtered date
plt.figure(figsize=(10, 6))
plt.plot(filtered_data['datetime'], y_test_filtered, color='blue', label='Actual BG')
plt.plot(filtered_data['datetime'], y_pred_filtered, color='red', label='Predicted BG')
plt.title('BG Prediction for 06-01-2022')
plt.xlabel('Date & Time')
plt.ylabel('BG (mg/dL)')
plt.legend()
plt.savefig('Exp3_part1_584.pdf')
plt.show()

###CG EGA code

In [ ]:
import numpy as np

def derivatives(y, freq):
    """
    Compute the derivative of the signal y, (y[i] - y[i-1]) / (t[i] - t[i-1]).
    :param y: signal, either true or predicted glucose values;
    :return: the derivatives and its associated signal (every sample has a corresponding derivative)
    """
    dy = np.reshape(np.diff(y, axis=1), (-1, 1)) / freq
    y = np.reshape(y[:, 1:], (-1, 1))
    return dy, y
"""
    This file contains AP/BE/EP filters for every glycemia regions. It is used by the CG-EGA object.
"""

filter_AP_hypo = [
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
]

filter_BE_hypo = [
    [0, 0, 0],
    [0, 0, 0],
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 0],
    [1, 0, 0],
    [0, 0, 0],
    [1, 0, 0],
]

filter_EP_hypo = [
    [0, 1, 1],
    [0, 1, 1],
    [0, 1, 1],
    [0, 1, 1],
    [1, 1, 1],
    [0, 1, 1],
    [1, 1, 1],
    [0, 1, 1],
]

filter_AP_eu = [
    [1, 1, 0],
    [1, 1, 0],
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
]

filter_BE_eu = [
    [0, 0, 0],
    [0, 0, 0],
    [1, 1, 0],
    [1, 1, 0],
    [1, 1, 0],
    [1, 1, 0],
    [0, 0, 0],
    [0, 0, 0],
]

filter_EP_eu = [
    [0, 0, 1],
    [0, 0, 1],
    [0, 0, 1],
    [0, 0, 1],
    [0, 0, 1],
    [0, 0, 1],
    [1, 1, 1],
    [1, 1, 1],
]

filter_AP_hyper = [
    [1, 1, 0, 0, 0],
    [1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
]

filter_BE_hyper = [
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0],
    [1, 1, 0, 0, 0],
    [1, 1, 0, 0, 0],
    [1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
]

filter_EP_hyper = [
    [0, 0, 1, 1, 1],
    [0, 0, 1, 1, 1],
    [0, 0, 1, 1, 1],
    [0, 0, 1, 1, 1],
    [0, 0, 1, 1, 1],
    [0, 0, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
]

In [ ]:
from datetime import datetime, timedelta, date
import pandas as pd
import numpy as np

def _any(l, axis=1):
    return np.reshape(np.any(np.concatenate(l, axis=axis), axis=axis), (-1, 1))


def _all(l, axis=1):
    return np.reshape(np.all(np.concatenate(l, axis=axis), axis=axis), (-1, 1))


def reshape_results(results, freq):
    # resampling
    if type(results.index) != type(date.today()): # in case the results objects isn't formatted with dates
        start_time = date.today().strftime("%Y-%m-%d %H:%M:%S")
        end_time = (datetime.now().replace(hour=0,minute=0,second=0,microsecond=0) + timedelta(minutes=freq*(len(results.index)-1))).strftime("%Y-%m-%d %H:%M:%S")
        index = pd.period_range(start=start_time, end=end_time, freq=str(freq) + 'min').to_timestamp()
        results.index = index
    else:
        start_time = datetime.strptime(results.index[0].strftime('%Y-%m-%d'), '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")
        end_time = (datetime.strptime(results.index[-1].strftime('%Y-%m-%d'), '%Y-%m-%d') + timedelta(days=1) - timedelta(
            minutes=float(freq))).strftime("%Y-%m-%d %H:%M:%S")
        index = pd.period_range(start=start_time, end=end_time, freq=str(freq) + 'min').to_timestamp()
        results = results.resample(str(freq) + 'min').mean()
        results = results.reindex(index=index)

    # creating the derivatives
    results = pd.concat([results, (results.diff(axis=0) / freq).rename(columns={"y_true": "dy_true", "y_pred": "dy_pred"})],
                        axis=1)

    return results


def extract_columns_from_results(results):
    return np.expand_dims(results.dropna().to_numpy().transpose(),axis=2)

In [ ]:
import numpy as np


class P_EGA():
    """
        The Point-Error Grid Analysis (P-EGA) gives an estimation of the clinical acceptability of the glucose
        predictions based on their point-accuracy. It is also known as the Clarke Error Grid Analysis (Clarke EGA).
        Every prediction is given a mark from A to E depending of the ground truth.

        This implementation follows "Evaluating the accuracy of continuous glucose-monitoring sensors: continuous
        glucose-error grid analysis illustrated by TheraSense Freestyle Navigator data.", Kovatchev et al., 2004.
    """

    def __init__(self, results, freq):
        """
        Instantiate the P-EGA object.
        :param results: dataframe with predictions and ground truths
        :param freq: prediction frequency in minutes (e.g., 5)
        """

        self.freq = freq
        self.results = reshape_results(results, self.freq)

    def full(self):
        """
            Full version of the P-EGA, which consists of an array giving for every prediction (row), its mark vector
            (column). There are 5 columns representing the mark A, B, C, D, and E.

            :return: numy array of shape (number of predictions, 5)
        """
        y_true, y_pred, dy_true, dy_pred = extract_columns_from_results(self.results)

        # if the true rate are big, we accept bigger mistake (region borders are modified)
        mod = np.zeros_like(y_true)
        mod[_any([
            _all([
                np.greater(dy_true, -2),
                np.less_equal(dy_true, -1)]),
            _all([
                np.less(dy_true, 2),
                np.greater_equal(dy_true, 1),
            ])
        ])] = 10

        mod[_any([
            _all([
                np.less_equal(dy_true, -2)
            ]),
            _all([
                np.greater_equal(dy_true, 2)
            ])
        ])] = 20

        A = _any([
            _all([
                np.less_equal(y_pred, 70 + mod),
                np.less_equal(y_true, 70)
            ]),
            _all([
                np.less_equal(y_pred, y_true * 6 / 5 + mod),
                np.greater_equal(y_pred, y_true * 4 / 5 - mod)
            ])
        ])

        E = _any([
            _all([
                np.greater(y_true, 180),
                np.less(y_pred, 70 - mod)
            ]),
            _all([
                np.greater(y_pred, 180 + mod),
                np.less_equal(y_true, 70)
            ])
        ])

        D = _any([
            _all([
                np.greater(y_pred, 70 + mod),
                np.greater(y_pred, y_true * 6 / 5 + mod),
                np.less_equal(y_true, 70),
                np.less_equal(y_pred, 180 + mod)
            ]),
            _all([
                np.greater(y_true, 240),
                np.less(y_pred, 180 - mod),
                np.greater_equal(y_pred, 70 - mod)
            ])
        ])

        C = _any([
            _all([
                np.greater(y_true, 70),
                np.greater(y_pred, y_true * 22 / 17 + (180 - 70 * 22 / 17) + mod)
            ]),
            _all([
                np.less_equal(y_true, 180),
                np.less(y_pred, y_true * 7 / 5 - 182 - mod)
            ])
        ])

        # B being the weirdest zone in the P-EGA, we compute it last by saying
        # it's all the points that have not been classified yet.
        B = _all([
            np.equal(A, False),
            np.equal(C, False),
            np.equal(D, False),
            np.equal(E, False),
        ])

        return np.concatenate([A, B, C, D, E], axis=1)

    def mean(self):
        return np.mean(self.full(), axis=0)

    def a_plus_b(self):
        full = self.full()
        a_plus_b = full[:, 0] + full[:, 1]
        return np.sum(a_plus_b) / len(a_plus_b)

In [ ]:
class R_EGA():
    """
        The Rate-Error Grid Analysis (P-EGA) gives an estimation of the clinical acceptability of the glucose
        predictions based on their rate-of-change-accuracy (the accuracy of the predicted variations).
        Every prediction is given a mark from {"A", "B", "uC", "lC", "uD", "lD", "uE", "lE"}

        The implementation is taken from "Evaluating the accuracy of continuous glucose-monitoring sensors: continuous
        glucose-error grid analysis illustrated by TheraSense Freestyle Navigator data.", Kovatchev et al., 2004.
    """

    def __init__(self, results, freq):
        """
        Instantiate the P-EGA object.
        :param results: dataframe with predictions and ground truths
        :param freq: prediction frequency in minutes (e.g., 5)
        """

        self.freq = freq
        self.results = reshape_results(results, self.freq)

    def full(self):
        """
            Full version of the R-EGA, which consists of an array giving for every prediction (row), its mark vector
            (column). There are 8 columns representing the mark A, B, uC, lC, uD, lD, uE, and lE.

            :return: numy array of shape (number of predictions, 8)
        """
        y_true, y_pred, dy_true, dy_pred = extract_columns_from_results(self.results)

        A = _any([
            _all([  # upper and lower
                np.greater_equal(dy_pred, dy_true - 1),
                np.less_equal(dy_pred, dy_true + 1)
            ]),
            _all([  # left
                np.less_equal(dy_pred, dy_true / 2),
                np.greater_equal(dy_pred, dy_true * 2)
            ]),
            _all([  # right
                np.less_equal(dy_pred, dy_true * 2),
                np.greater_equal(dy_pred, dy_true / 2)
            ])
        ])

        B = _all([
            np.equal(A, False),  # not in A but satisfies the cond below
            _any([
                _all([
                    np.less_equal(dy_pred, -1),
                    np.less_equal(dy_true, -1)
                ]),
                _all([
                    np.less_equal(dy_pred, dy_true + 2),
                    np.greater_equal(dy_pred, dy_true - 2)
                ]),
                _all([
                    np.greater_equal(dy_pred, 1),
                    np.greater_equal(dy_true, 1)
                ])
            ])
        ])

        uC = _all([
            np.less(dy_true, 1),
            np.greater_equal(dy_true, -1),
            np.greater(dy_pred, dy_true + 2)
        ])

        lC = _all([
            np.less_equal(dy_true, 1),
            np.greater(dy_true, -1),
            np.less(dy_pred, dy_true - 2)
        ])

        uD = _all([
            np.less_equal(dy_pred, 1),
            np.greater_equal(dy_pred, -1),
            np.greater(dy_pred, dy_true + 2)
        ])

        lD = _all([
            np.less_equal(dy_pred, 1),
            np.greater_equal(dy_pred, -1),
            np.less(dy_pred, dy_true - 2)
        ])

        uE = _all([
            np.greater(dy_pred, 1),
            np.less(dy_true, -1)
        ])

        lE = _all([
            np.less(dy_pred, -1),
            np.greater(dy_true, 1)
        ])

        return np.concatenate([A, B, uC, lC, uD, lD, uE, lE], axis=1)

    def mean(self):
        return np.mean(self.full(), axis=0)

    def a_plus_b(self):
        full = self.full()
        a_plus_b = full[:, 0] + full[:, 1]
        return np.sum(a_plus_b) / len(a_plus_b)

In [ ]:
class CG_EGA():
    """
        The Continuous Glucose-Error Grid Analysis (CG-EGA) gives a measure of the clinical acceptability of the glucose predictions. It analyzes both the
        prediction accuracy (through the P-EGA) and the predicted variation accuracy (R-EGA).

        The implementation has been made following "Evaluating the accuracy of continuous glucose-monitoring sensors:
        continuous glucose-error grid analysis illustrated by TheraSense Freestyle Navigator data.", Kovatchev et al., 2004.
    """

    day_len = 1440

    def __init__(self, results, freq):
        """
        Instantiate the CG-EGA object.
        :param results: dataframe with predictions and ground truths
        :param freq: prediction frequency in minutes (e.g., 5)
        """
        self.results = reshape_results(results, freq)
        self.freq = freq
        self.day_len = self.day_len // freq
        self.p_ega = P_EGA(results, freq).full()
        self.r_ega = R_EGA(results, freq).full()


    def full(self):
        """
            Full version of the CG-EGA, which consists of 3 tables (representing the hypoglycemia, euglycemia, and
            hyperglycemia regions) being the cartesian product between the P-EGA and the R-EGA. Every cell contains the
            number of predictions falling into it.

            :return: hypoglycemia full CG-EGA, euglycemia full CG-EGA, hyperglycemia full CG-EGA
        """
        y_true, y_pred, dy_true, dy_pred = extract_columns_from_results(self.results)
        p_ega, r_ega = self.p_ega, self.r_ega

        # compute the glycemia regions
        hypoglycemia = np.less_equal(y_true, 70).reshape(-1, 1)
        euglycemia = _all([
            np.less_equal(y_true, 180),
            np.greater(y_true, 70)
        ]).reshape(-1, 1)
        hyperglycemia = np.greater(y_true, 180).reshape(-1, 1)

        # apply region filter and convert to 0s and 1s
        P_hypo = np.reshape(np.concatenate([np.reshape(p_ega[:, 0], (-1, 1)),
                                            np.reshape(p_ega[:, 3], (-1, 1)),
                                            np.reshape(p_ega[:, 4], (-1, 1))], axis=1).astype("int32") *
                            hypoglycemia.astype("int32"),
                            (-1, 3))
        P_eu = np.reshape(np.concatenate([np.reshape(p_ega[:, 0], (-1, 1)),
                                          np.reshape(p_ega[:, 1], (-1, 1)),
                                          np.reshape(p_ega[:, 2], (-1, 1))], axis=1).astype("int32") *
                          euglycemia.astype("int32"),
                          (-1, 3))
        P_hyper = np.reshape(p_ega.astype("int32") * hyperglycemia.astype("int32"), (-1, 5))

        R_hypo = np.reshape(r_ega.astype("int32") * hypoglycemia.astype("int32"), (-1, 8))
        R_eu = np.reshape(r_ega.astype("int32") * euglycemia.astype("int32"), (-1, 8))
        R_hyper = np.reshape(r_ega.astype("int32") * hyperglycemia.astype("int32"), (-1, 8))

        CG_EGA_hypo = np.dot(np.transpose(R_hypo), P_hypo)
        CG_EGA_eu = np.dot(np.transpose(R_eu), P_eu)
        CG_EGA_hyper = np.dot(np.transpose(R_hyper), P_hyper)

        return CG_EGA_hypo, CG_EGA_eu, CG_EGA_hyper

    def simplified(self, count=False):
        """
            Simplifies the full CG-EGA into Accurate Prediction (AP), Benign Prediction (BE), and Erroneous Prediction (EP)
            rates for every glycemia regions.

            :param count: if False, the results, for every region, will be expressed as a ratio

            :return: AP rate in hypoglycemia, BE rate in hypoglycemia, EP rate in hypoglycemia,
                     AP rate in euglycemia, BE rate in euglycemia, EP rate in euglycemia,
                     AP rate in hyperglycemia, BE rate in hyperglycemia, EP rate in hyperglycemia
        """

        CG_EGA_hypo, CG_EGA_eu, CG_EGA_hyper = self.full()

        AP_hypo = np.sum(CG_EGA_hypo * filter_AP_hypo)
        BE_hypo = np.sum(CG_EGA_hypo * filter_BE_hypo)
        EP_hypo = np.sum(CG_EGA_hypo * filter_EP_hypo)

        AP_eu = np.sum(CG_EGA_eu * filter_AP_eu)
        BE_eu = np.sum(CG_EGA_eu * filter_BE_eu)
        EP_eu = np.sum(CG_EGA_eu * filter_EP_eu)

        AP_hyper = np.sum(CG_EGA_hyper * filter_AP_hyper)
        BE_hyper = np.sum(CG_EGA_hyper * filter_BE_hyper)
        EP_hyper = np.sum(CG_EGA_hyper * filter_EP_hyper)

        if not count:
            sum_hypo = (AP_hypo + BE_hypo + EP_hypo)
            sum_eu = (AP_eu + BE_eu + EP_eu)
            sum_hyper = (AP_hyper + BE_hyper + EP_hyper)


            [AP_hypo, BE_hypo, EP_hypo] = [AP_hypo / sum_hypo, BE_hypo / sum_hypo, EP_hypo / sum_hypo] if not sum_hypo == 0 else [np.nan, np.nan, np.nan]
            [AP_eu, BE_eu, EP_eu] = [AP_eu / sum_eu, BE_eu / sum_eu, EP_eu / sum_eu] if not sum_eu == 0 else [np.nan, np.nan, np.nan]
            [AP_hyper, BE_hyper, EP_hyper] = [AP_hyper / sum_hyper, BE_hyper / sum_hyper, EP_hyper / sum_hyper] if not sum_hyper == 0 else [np.nan, np.nan, np.nan]

        return AP_hypo, BE_hypo, EP_hypo, AP_eu, BE_eu, EP_eu, AP_hyper, BE_hyper, EP_hyper

    def reduced(self):
        """
            Reduces the simplified CG-EGA by not dividing the results into the glycemia regions
            :return: overall AP rate, overall BE rate, overall EP rate
        """

        AP_hypo, BE_hypo, EP_hypo, AP_eu, BE_eu, EP_eu, AP_hyper, BE_hyper, EP_hyper = self.simplified(count=True)
        sum = (AP_hypo + BE_hypo + EP_hypo + AP_eu + BE_eu + EP_eu + AP_hyper + BE_hyper + EP_hyper)
        return (AP_hypo + AP_eu + AP_hyper) / sum, (BE_hypo + BE_eu + BE_hyper) / sum, (
                EP_hypo + EP_eu + EP_hyper) / sum

    def per_sample(self):
        """
            Compute the per-sample simplified CG-EGA
            :return: pandas DataFrame with columns (y_true, y_pred, dy_true, dy_pred, P-EGA mark, R-EGA mark,
                                                    CG-EGA AP/BE/EP mark)
        """

        y_true, y_pred, dy_true, dy_pred = extract_columns_from_results(self.results)
        p_ega, r_ega = self.p_ega, self.r_ega

        df = pd.DataFrame(data=np.c_[y_true, dy_true, y_pred, dy_pred, p_ega, r_ega])
        df["CG_EGA"] = len(df.index) * ["?"]

        df.columns = ["y_true", "dy_true", "y_pred", "dy_pred", "P_A", "P_B", "P_C", "P_D", "P_E", "R_A", "R_B", "R_uC",
                      "R_lC", "R_uD", "R_lD", "R_uE", "R_lE", "CG_EGA"]

        p_ega_labels = ["A", "B", "C", "D", "E"]
        r_ega_labels = ["A", "B", "uC", "lC", "uD", "lD", "uE", "lE"]

        df["time"] = (np.arange(len(df.index)) + 1) * self.freq

        for i in range(len(p_ega)):
            p_ega_i = df.iloc[i, 4:9].values.reshape(1, -1)
            r_ega_i = df.iloc[i, 9:17].values.reshape(1, -1)
            y_true_i = df.iloc[i, 0]

            cg_ega_i = np.dot(np.transpose(r_ega_i), p_ega_i)

            if y_true_i <= 70:
                # hypoglycemia
                cg_ega_i = cg_ega_i[:, [0, 3, 4]]

                if np.sum(cg_ega_i * filter_AP_hypo) == 1:
                    df.loc[i, "CG_EGA"] = "AP"
                elif np.sum(cg_ega_i * filter_BE_hypo) == 1:
                    df.loc[i, "CG_EGA"] = "BE"
                else:
                    df.loc[i, "CG_EGA"] = "EP"

            elif y_true_i <= 180:
                # euglycemia
                cg_ega_i = cg_ega_i[:, [0, 1, 2]]

                if np.sum(cg_ega_i * filter_AP_eu) == 1:
                    df.loc[i, "CG_EGA"] = "AP"
                elif np.sum(cg_ega_i * filter_BE_eu) == 1:
                    df.loc[i, "CG_EGA"] = "BE"
                else:
                    df.loc[i, "CG_EGA"] = "EP"
            else:
                # hyperglycemia
                if np.sum(cg_ega_i * filter_AP_hyper) == 1:
                    df.loc[i, "CG_EGA"] = "AP"
                elif np.sum(cg_ega_i * filter_BE_hyper) == 1:
                    df.loc[i, "CG_EGA"] = "BE"
                else:
                    df.loc[i, "CG_EGA"] = "EP"

            df.loc[i, "P_EGA"] = p_ega_labels[np.argmax(p_ega_i.ravel())]
            df.loc[i, "R_EGA"] = r_ega_labels[np.argmax(r_ega_i.ravel())]

        df.index = pd.notnull(self.results).all(1).to_numpy().nonzero()[0]
        df_nan = pd.concat([
            self.results.copy().reset_index().rename(columns={"index": "datetime"}),
            df.loc[:, ["CG_EGA", "P_EGA", "R_EGA"]]
        ], axis=1)

        return df_nan

    def plot(self, day=0):
        """
        Plot the given day predictions and CG-EGA
        :param day: (int) number of the day for which to plot the predictions and CG-EGA
        :return: /
        """
        res = self.per_sample().iloc[day * self.day_len:(day + 1) * self.day_len - 1]
        pd.plotting.register_matplotlib_converters()
        ap = res[res["CG_EGA"] == "AP"]
        be = res[res["CG_EGA"] == "BE"]
        ep = res[res["CG_EGA"] == "EP"]

        f = plt.figure(figsize=(10, 4.5))
        # ax1 = f.add_subplot(211)
        ax2 = f.add_subplot(121)
        ax3 = f.add_subplot(122)

        # prediction VS truth against time
        # ax1.plot(res["datetime"], res["y_true"], "k", label="y_true")
        # ax1.plot(res["datetime"], res["y_pred"], "--k", label="y_pred")
        # ax1.plot(ap["datetime"], ap["y_pred"], label="AP", marker="o", mec="xkcd:green", mfc="xkcd:green", ls="")
        # ax1.plot(be["datetime"], be["y_pred"], label="BE", marker="o", mec="xkcd:orange", mfc="xkcd:orange", ls="")
        # ax1.plot(ep["datetime"], ep["y_pred"], label="EP", marker="o", mec="xkcd:red", mfc="xkcd:red", ls="")
        # ax1.set_title("Prediction VS ground truth as a function of time")
        # ax1.set_xlabel("Time (min)")
        # ax1.set_ylabel("Glucose value (mg/dL)")
        # ax1.legend()

        # P-EGA structure
        ax2.plot([0, 400], [0, 400], "-k", linewidth=0.75)
        ax2.plot([58.33, 400], [58.33333 * 6 / 5, 400 * 6 / 5], "-k")
        ax2.plot([0, 58.33333], [70, 70], "-k")
        ax2.plot([70, 400], [56, 320], "-k")
        ax2.plot([70, 70], [0, 56], "-k")
        ax2.plot([70, 70], [84, 400], "-k")
        ax2.plot([0, 70], [180, 180], "-k")
        ax2.plot([70, 400], [70 * 22 / 17 + 89.412, 400 * 22 / 17 + 89.412], "-k")
        ax2.plot([180, 180], [0, 70], "-k")
        ax2.plot([180, 400], [70, 70], "-k")
        ax2.plot([240, 240], [70, 180], "-k")
        ax2.plot([240, 400], [180, 180], "-k")
        ax2.plot([130, 180], [130 * 7 / 5 - 182, 180 * 7 / 5 - 182], "-k")
        ax2.plot([130, 180], [130 * 7 / 5 - 202, 180 * 7 / 5 - 202], "--k")
        ax2.plot([180, 400], [50, 50], "--k")
        ax2.plot([240, 400], [160, 160], "--k")
        ax2.plot([58.33333, 400], [58.33333 * 6 / 5 + 20, 400 * 6 / 5 + 20], "--k")
        ax2.plot([0, 58.33333], [90, 90], "--k")
        ax2.plot([0, 70], [200, 200], "--k")
        ax2.plot([70, 400], [70 * 22 / 17 + 109.412, 400 * 22 / 17 + 109.412], "--k")

        ax2.text(38, 12, "A")
        ax2.text(12, 38, "A")
        ax2.text(375, 240, "B")
        ax2.text(260, 375, "B")
        ax2.text(150, 375, "C")
        ax2.text(165, 25, "C")
        ax2.text(25, 125, "D")
        ax2.text(375, 125, "D")
        ax2.text(375, 25, "E")
        ax2.text(25, 375, "E")

        ax2.set_xlim(0, 400)
        ax2.set_ylim(0, 400)

        ax2.set_xlabel("True glucose value [mg/dL]")
        ax2.set_ylabel("Predicted glucose value [mg/dL]")
        ax2.set_title("Point-Error Grid Analysis")

        # P-EGA data
        ax2.plot(ap["y_true"], ap["y_pred"], label="AP", marker="o", mec="xkcd:green", mfc="xkcd:green", ls="")
        ax2.plot(be["y_true"], be["y_pred"], label="BE", marker="o", mec="xkcd:orange", mfc="xkcd:orange", ls="")
        ax2.plot(ep["y_true"], ep["y_pred"], label="EP", marker="o", mec="xkcd:red", mfc="xkcd:red", ls="")

        ax2.legend()

        # R-EGA structure
        ax3.plot([-4, 4], [-4, 4], "-k", linewidth=0.75)
        ax3.plot([-4, -1, -1], [1, 1, 4], "-k")
        ax3.plot([-4, -3, 1, 1], [-1, -1, 3, 4], "-k")
        ax3.plot([-4, -2, 1, 2], [-2, -1, 2, 4], "-k")
        ax3.plot([-2, -1, 2, 4], [-4, -2, 1, 2], "-k")
        ax3.plot([-1, -1, 3, 4], [-4, -3, 1, 1], "-k")
        ax3.plot([1, 1, 4], [-4, -1, -1], "-k")

        ax3.text(-3.25, -3.75, "A")
        ax3.text(-3.75, -3.25, "A")
        ax3.text(-1.35, -3.5, "B")
        ax3.text(-3.5, -1.35, "B")
        ax3.text(0, -3.5, "C")
        ax3.text(0, 3.5, "C")
        ax3.text(-3.5, 0.5, "D")
        ax3.text(3.5, -0.5, "D")
        ax3.text(-3.5, 3.5, "E")
        ax3.text(3.5, -3.5, "E")

        ax3.set_xlim(-4, 4)
        ax3.set_ylim(-4, 4)
        ax3.set_xlabel("True glucose rate of change [mg/dL/min]")
        ax3.set_ylabel("Predicted glucose rate of change [mg/dL/min]")
        ax3.set_title("Rate-Error Grid Analysis")

        # R-EGA data
        ax3.plot(ap["dy_true"], ap["dy_pred"], label="AP", marker="o", mec="xkcd:green", mfc="xkcd:green", ls="")
        ax3.plot(be["dy_true"], be["dy_pred"], label="BE", marker="o", mec="xkcd:orange", mfc="xkcd:orange", ls="")
        ax3.plot(ep["dy_true"], ep["dy_pred"], label="EP", marker="o", mec="xkcd:red", mfc="xkcd:red", ls="")

        ax3.legend()
        plt.savefig('Exp3_part1_cgega_584.pdf')
        plt.show()

###CG EGA Code Call

In [ ]:
freq = 5
results = pd.DataFrame(data = np.c_[y_test.reshape(-1,1),y_pred.reshape(-1,1)], columns=["y_true","y_pred"])
cg_ega = CG_EGA(results, freq)
print("AP, BE, EP:", cg_ega.reduced())
cg_ega.plot(day=0)